# The Boston Housing Price dataset

In [1]:
from keras.datasets import boston_housing

from keras import models
from keras import layers

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


### load data

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

### eda

In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
train_data[:2]

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        6.14200e+00, 9.17000e+01, 3.97690e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, 0.00000e+00, 4.15000e-01,
        7.61000e+00, 1.57000e+01, 6.27000e+00, 2.00000e+00, 3.48000e+02,
        1.47000e+01, 3.95380e+02, 3.11000e+00]])

In [6]:
train_targets[:2]

array([15.2, 42.3])

In [7]:
# features: 
# 13 numerical features, such as per capita crime rate, average number of rooms per dwelling, 
# accessibility to highways, and so on.

# Preparing the data (normalisation)

In [8]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

Note: for calculating mean and std we used oly the training!!!

# Building your network

In [9]:
train_data.shape[1]

13

In [10]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    # mse: mean squared error
    # mae: mean absolute error
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    return model

# K-fold validation

In [11]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100

In [ ]:
%%time

all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(   [train_data[:i * num_val_samples], 
                                            train_data[(i + 1) * num_val_samples:]],
                                        axis=0)
    
    partial_train_targets = np.concatenate(   [train_targets[:i * num_val_samples],
                                               train_targets[(i + 1) * num_val_samples:]],
                                           axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    
    #loss and accuracy
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2


In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

In [ ]:
np.std(all_scores)

In [ ]:
train_targets.min(), train_targets.max()

# different approach to save partial mae 

In [ ]:
num_epochs = 400

In [ ]:
%%time

all_mae_histories = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(   [train_data[:i * num_val_samples], 
                                            train_data[(i + 1) * num_val_samples:]],
                                        axis=0)
    
    partial_train_targets = np.concatenate(   [train_targets[:i * num_val_samples],
                                               train_targets[(i + 1) * num_val_samples:]],
                                           axis=0)
    model = build_model()
    
    history = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    
   
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

# smoothing the plot

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
### remove also the first 10 points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])

In [ ]:
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

# Final model

In [ ]:
model = build_model()

In [ ]:
%%time

model.fit(train_data, train_targets, epochs=310, batch_size=128, verbose=0)

### evaluate on test

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score